In [ ]:
from . import models
from .database import engine

models.Base.metadata.create_all(bind=engine)


# from app.models import Document, DocumentStatus
# from app.database import SessionLocal
# 
# db = SessionLocal()
# new_doc = Document(
#     filename="example.pdf",
#     filehash="abc123hash",
#     filepath="/files/example.pdf",
#     status=DocumentStatus.NEW
# )
# db.add(new_doc)
# db.commit()
# db.refresh(new_doc)


In [ ]:
from tools.document_loader import Text_Document
from tools.pdf_tool import load_pdf
from tools.slack import slack_message_retriever
from tools.ocr import image_to_text
from tools.video import transcribe_video
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing_extensions import TypedDict
from typing import Annotated
from langchain_core.messages import AnyMessage, ToolMessage, HumanMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain.retrievers.document_compressors import LLMListwiseRerank
from IPython.display import Image, display
import os
import json

In [ ]:
load_dotenv()    
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [ ]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def make_tools_calling_llm(llm_with_tools):
    def tools_calling_llm(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}
    return tools_calling_llm

In [ ]:
llm = ChatGroq(model="qwen-qwq-32b")
pdf_tool = load_pdf("attention.pdf")
text_tool = Text_Document("speech.txt")
slack_tool = slack_message_retriever(limit=10)
tools = [pdf_tool, text_tool, slack_tool]
llm_with_tools = llm.bind_tools(tools=tools)

builder = StateGraph(State)
builder.add_node("tools_calling_llm", make_tools_calling_llm(llm_with_tools))
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "tools_calling_llm")
builder.add_conditional_edges("tools_calling_llm", tools_condition)
builder.add_edge("tools", END)
graph = builder.compile()

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({
    "messages": [HumanMessage(content="Who are the authors of attention is all you need")]
})

for m in response['messages']:
    if isinstance(m, ToolMessage):
        try:
            content_data = json.loads(m.content)
            print("Source:", content_data[0]["metadata"]["source"]) 
            print(content_data)
        except Exception as e:
            print("Error parsing tool message:", e) 
    else:
        print("Assistant:", m.content)